## Building a Simple Perceptron (Logistic Regression), using NN module

In this section, we are going to build a **simple Perceptron model** that uses:

- **Sigmoid activation function**  
- **Binary Cross-Entropy (BCE) loss function**

This setup is essentially equivalent to a **Logistic Regression model**.  

We will use the **Titanic** to train and evaluate our model.


In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn

# 1) Dataset

In [2]:
data = sns.load_dataset('titanic')
data.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [3]:
# Selecting useful features

df = data[["survived", "pclass", "sex", "age", "fare", "embarked"]].copy()
df.head()

,survived,pclass,sex,age,fare,embarked
0,0,3,male,22.0,7.2500,S
1,1,1,female,38.0,71.2833,C
2,1,3,female,26.0,7.9250,S
3,1,1,female,35.0,53.1000,S
4,0,3,male,35.0,8.0500,S


## 1.1) Handling the missing values

In [4]:
df.isnull().sum()

,0
survived,0
pclass,0
sex,0
age,177
fare,0
embarked,2


In [5]:
# Filling missing age value with median of age
median_age = df['age'].median()
df['age'] = df['age'].fillna(median_age)

# Filling missing embarked value with mode
mode_embarked = df['embarked'].mode()[0]
df['embarked'] = df['embarked'].fillna(mode_embarked)

## 1.2) Encoding categorical features

- sex → binary encoding (male=0, female=1)

- embarked → one-hot encoding (C, Q, S → separate columns)

In [6]:
# Binary encoding for sex
df['sex'] = df['sex'].map({'male': 0, 'female': 1})

# One-hot encoding for embarked
df = pd.get_dummies(df, columns=['embarked'], dtype = int)

df.head()

,survived,pclass,sex,age,fare,embarked_C,embarked_Q,embarked_S
0,0,3,0,22.0,7.2500,0,0,1
1,1,1,1,38.0,71.2833,1,0,0
2,1,3,1,26.0,7.9250,0,0,1
3,1,1,1,35.0,53.1000,0,0,1
4,0,3,0,35.0,8.0500,0,0,1


## 1.3) Train test split

In [7]:
# Defining X(input) and Y(output)

x = df.drop('survived', axis=1)
y = df['survived']

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

## 1.4) Scaling age & fare column

In [9]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train[['age', 'fare']] = scaler.fit_transform(x_train[['age', 'fare']])
x_test[['age', 'fare']] = scaler.transform(x_test[['age', 'fare']])

x_train.head()

,pclass,sex,age,fare,embarked_C,embarked_Q,embarked_S
331,1,0,1.253641,-0.078684,0,0,1
733,2,0,-0.477284,-0.377145,0,0,1
382,3,0,0.215086,-0.474867,0,0,1
704,3,0,-0.246494,-0.476230,0,0,1
813,3,1,-1.785093,-0.025249,0,0,1


# 2) Converting Numpy arrays to PyTroch tensors

In [10]:
# Converting DataFrames to NumPy arrays
x_train = x_train.to_numpy()
x_test = x_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

# Converting numpy arrays to tensors
x_train = torch.from_numpy(x_train).float()
x_test = torch.from_numpy(x_test).float()
y_train = torch.from_numpy(y_train).long()                       # Labels are categorial, that's why we use long(integer)
y_test = torch.from_numpy(y_test).long()

print(type(x_train))
print(type(y_train))
print(type(x_test))
print(type(y_test))

<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>


In [11]:
# First 5 data points
print(x_train[:5])
print(y_train[:5])

tensor([[ 1.0000,  0.0000,  1.2536, -0.0787,  0.0000,  0.0000,  1.0000],
        [ 2.0000,  0.0000, -0.4773, -0.3771,  0.0000,  0.0000,  1.0000],
        [ 3.0000,  0.0000,  0.2151, -0.4749,  0.0000,  0.0000,  1.0000],
        [ 3.0000,  0.0000, -0.2465, -0.4762,  0.0000,  0.0000,  1.0000],
        [ 3.0000,  1.0000, -1.7851, -0.0252,  0.0000,  0.0000,  1.0000]])
tensor([0, 0, 0, 0, 0])


# 3) Defining the perceptron model

In [12]:
class perceptron(nn.Module):

  def __init__(self, input_data):

    super(perceptron, self).__init__()                             # Custom neural n/w (percetron) inherits from nn.Module. So we call the parent class constructor
                                                                   # (nn.Module.__init__) to properly initialize the base class
    num_features = input_data.shape[1]
    self.layer1 = nn.Linear(num_features, 1)                       # nn.Linear(in_features, out_features)
    self.activation1 = nn.Sigmoid()

  def forward(self, input_data):

    z = self.layer1(input_data)                                     # Linear layer calculates weighted sum(z) = Weight*input_feature + bias
    y_pred = self.activation1(z)
    y_pred = y_pred.view(-1)                                        # from [712, 1] → [712]; from [batch_size, 1] → [batch_size]
    return y_pred


  def loss_function(self, y_pred, y):

    loss = nn.BCELoss()
    loss =  loss(y_pred, y.float())
    return loss

# 4) Training Perceptron model on the dataset(Training pipeline)

In [13]:
# Create a model
model = perceptron(x_train)
print(model)

# Creating optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

# Training model
for epoch in range(25):

  # Forward pass
  y_pred = model.forward(x_train)

  # Backward pass
  loss = model.loss_function(y_pred, y_train)                 # Calculating loss
  loss.backward()                                             # Computes Gradients

  # Update parameters
  optimizer.step()

  # Reset gradients
  optimizer.zero_grad()

  print(f"Epoch {epoch+1}: loss = {loss.item()}")

perceptron(
  (layer1): Linear(in_features=7, out_features=1, bias=True)
  (activation1): Sigmoid()
)
Epoch 1: loss = 0.836203396320343
Epoch 2: loss = 0.7648965120315552
Epoch 3: loss = 0.7155475616455078
Epoch 4: loss = 0.6817837357521057
Epoch 5: loss = 0.6585845351219177
Epoch 6: loss = 0.6423789262771606
Epoch 7: loss = 0.6307645440101624
Epoch 8: loss = 0.6221696138381958
Epoch 9: loss = 0.6155760884284973
Epoch 10: loss = 0.6103252172470093
Epoch 11: loss = 0.6059894561767578
Epoch 12: loss = 0.602288544178009
Epoch 13: loss = 0.599037766456604
Epoch 14: loss = 0.5961141586303711
Epoch 15: loss = 0.5934349298477173
Epoch 16: loss = 0.5909435153007507
Epoch 17: loss = 0.588601291179657
Epoch 18: loss = 0.5863810181617737
Epoch 19: loss = 0.5842633843421936
Epoch 20: loss = 0.582234263420105
Epoch 21: loss = 0.5802834033966064
Epoch 22: loss = 0.5784028768539429
Epoch 23: loss = 0.5765863656997681
Epoch 24: loss = 0.5748290419578552
Epoch 25: loss = 0.5731266140937805


In [14]:
# Looking at the updates parameters:
print(model.layer1.weight)
print(model.layer1.bias)

Parameter containing:
tensor([[-0.2559,  0.5038, -0.1140,  0.0527,  0.3508, -0.2375,  0.0986]],
       requires_grad=True)
Parameter containing:
tensor([-0.3110], requires_grad=True)


# 5) Model Evaluation

In [15]:
with torch.no_grad():
  # Predicting values on testing data
  y_test_pred = model.forward(x_test)
  print(y_test_pred)

  # Converting probability values to predicted class
  y_pred = (y_test_pred > 0.8).float()
  print(y_pred)

tensor([0.3241, 0.3181, 0.2840, 0.4959, 0.4719, 0.5276, 0.3043, 0.2933, 0.3270,
        0.5296, 0.3738, 0.2433, 0.3839, 0.3185, 0.3092, 0.5395, 0.3635, 0.3043,
        0.3263, 0.3533, 0.2770, 0.3737, 0.3984, 0.2843, 0.2713, 0.3065, 0.4217,
        0.3187, 0.3178, 0.3735, 0.2858, 0.3921, 0.4483, 0.3796, 0.2876, 0.3258,
        0.3882, 0.3043, 0.5693, 0.2700, 0.3519, 0.2681, 0.2700, 0.2090, 0.4059,
        0.2968, 0.2859, 0.2790, 0.2768, 0.4045, 0.4936, 0.4744, 0.2536, 0.4452,
        0.1604, 0.5452, 0.3162, 0.5802, 0.4297, 0.3213, 0.2822, 0.4771, 0.4461,
        0.3859, 0.2090, 0.3807, 0.3519, 0.2683, 0.3437, 0.5335, 0.4504, 0.6500,
        0.4525, 0.5629, 0.2787, 0.2579, 0.2996, 0.5910, 0.4213, 0.3561, 0.2979,
        0.4400, 0.5736, 0.2090, 0.3886, 0.3623, 0.5875, 0.5758, 0.3593, 0.2699,
        0.3068, 0.4254, 0.3214, 0.2090, 0.2700, 0.2700, 0.3513, 0.2417, 0.4256,
        0.2717, 0.3143, 0.2579, 0.6074, 0.2529, 0.2657, 0.2752, 0.4234, 0.3501,
        0.2669, 0.4225, 0.5164, 0.2830, 

In [16]:
# Accuracy
num_correct_pred = (y_pred == y_test).sum().item()
accuracy = num_correct_pred / len(y_test)

print("Test Accuracy:", accuracy * 100, "%")

Test Accuracy: 58.659217877094974 %
